<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_10_01_Compare_Multimodal_Image_Descriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a series of images, and compare the quality of different multimodal models for image descriptions.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
from typing import List, Optional

async def create_openai_specification(model: enums.OpenAIModels, prompt: str, temperature: Optional[float] = None):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{model}]",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        customInstructions=prompt,
        openAI=input_types.OpenAIModelPropertiesInput(
            temperature=temperature,
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_anthropic_specification(model: enums.AnthropicModels, prompt: str, temperature: Optional[float] = None):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Anthropic [{model}]",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        customInstructions=prompt,
        anthropic=input_types.AnthropicModelPropertiesInput(
            temperature=temperature,
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_mistral_specification(model: enums.MistralModels, prompt: str, temperature: Optional[float] = None):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Mistral [{model}]",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.MISTRAL,
        customInstructions=prompt,
        mistral=input_types.MistralModelPropertiesInput(
            temperature=temperature,
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_google_specification(model: enums.GoogleModels, prompt: str, temperature: Optional[float] = None):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Google [{model}]",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.GOOGLE,
        customInstructions=prompt,
        google=input_types.GoogleModelPropertiesInput(
            temperature=temperature,
            model=model
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Image Extraction",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_IMAGE,
                        modelImage=input_types.ModelImageExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def ingest_uri(uri: str, workflow_id: str):
    if graphlit.client is None:
        return

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, workflow=input_types.EntityReferenceInput(id=workflow_id), is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def get_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.get_content(content_id)

        return response.content
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_content(content_id: str):
    if graphlit.client is None:
        return;

    try:
        _ = await graphlit.client.delete_content(content_id)

        return
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [4]:
from IPython.display import display, Markdown

# Remove any existing specifications, workflows and contents; only needed for notebook example
await delete_all_specifications()
await delete_all_workflows()
await delete_all_contents()

print('Deleted all specifications, workflows and contents.')

# Specify image to be analyzed
uri = "https://graphlitplatform.blob.core.windows.net/test/images/medical_chart.jpeg"

# Specify the extraction prompt to be applied to the image
prompt = "Summarize this image."
#prompt = "Please give concise summary to be added in document. No explanation."

# Specify the model temperature
temperature = 0.3

specification_id = await create_anthropic_specification(enums.AnthropicModels.CLAUDE_3_5_SONNET, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with Anthropic Claude 3.5 Sonnet'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)

specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_128K, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with OpenAI GPT-4o'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)

specification_id = await create_openai_specification(enums.OpenAIModels.GPT4O_MINI_128K, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with OpenAI GPT-4o Mini'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)

specification_id = await create_mistral_specification(enums.MistralModels.PIXTRAL_12B_2409, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with Mistral Pixtral'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)

specification_id = await create_google_specification(enums.GoogleModels.GEMINI_1_5_FLASH_002, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with Google Gemini 1.5 Flash'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)

specification_id = await create_google_specification(enums.GoogleModels.GEMINI_1_5_PRO_002, prompt, temperature)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        content_id = await ingest_uri(uri, workflow_id)

        if content_id is not None:
            print(f'Ingested content [{content_id}].')

            content = await get_content(content_id)

            if content is not None and content.image is not None:
                display(Markdown(f'### Described image [{content.file_name}] with Google Gemini 1.5 Pro'))

                if content.image.description is not None:
                    display(Markdown(content.image.description))
                else:
                    print('- No description generated')

            await delete_content(content_id)


Deleted all specifications, workflows and contents.
Created specification [2e90fe8e-4d11-4614-a56b-1398326efba0].
Created workflow [7a09fbb7-98bd-45fb-b065-d5d09d851614].
Ingested content [476c4ac7-5b9f-44b2-acac-0fbda9d81cc4].


### Described image [medical_chart.jpeg] with Anthropic Claude 3.5 Sonnet

This image shows a statistical analysis graph of SE-HPLC (% Main Peak) purity data for a drug product stored at a recommended temperature. The graph plots the purity percentage on the y-axis against time in months on the x-axis, up to 48 months.

The graph includes raw data points, represented by circles, which mostly cluster between 99.4% and 99.8% purity. There are three trend lines: a blue line representing the predicted mean, a green dashed line showing the 1-sided 95% confidence bound for the worst-case lot, and a red line indicating the worst-case lot 049D108163.

A horizontal red line at 98.5% marks the specification limit. The graph also indicates the current shelf life with a vertical dashed line around 36 months. Overall, the data suggests that the drug product maintains high purity levels well above the specification throughout the observed time period.

Created specification [968ed675-132b-42a3-8ee2-6b61e4f868a2].
Created workflow [606c4231-9dd7-4050-8bb2-00f19e16b781].
Ingested content [dd72d8ae-7563-4bfd-a5e0-820fb864d525].


### Described image [medical_chart.jpeg] with OpenAI GPT-4o

...

### Described image [medical_chart.jpeg] with Google Gemini 1.5 Flash

Figure 1 shows a statistical analysis of SE-HPLC (% Main Peak) purity data for a drug product stored at the recommended temperature.  The graph displays raw data, the worst-case lot data, a 95% confidence bound, the current shelf life, and the predicted mean purity over time (in months).

Created specification [c67cba31-defd-4d17-bb8d-ffabaef915d7].
Created workflow [6201a6b0-b020-4837-912d-f8af73b38348].
Ingested content [720c8837-d656-4fee-b214-e421148e0436].


### Described image [medical_chart.jpeg] with Google Gemini 1.5 Pro

This figure presents a statistical analysis of SE-HPLC purity data, specifically the percentage of the main peak, for a drug product stored at a recommended temperature. The x-axis represents time in months, ranging from 0 to 48, while the y-axis represents the percentage of the main peak by SE-HPLC, ranging from approximately 98.5 to 100. The data points are represented by open circles, with the worst-case lot highlighted by hash symbols.  A specification limit is indicated at 98.5%. The graph also includes a predicted mean line, a current shelf life line, and a 1-sided 95% confidence bound on the worst-case lot.